This implementation is intended to explain the stages in which the hash function goes through.

In [1]:
# create three different messages
input_1 = "message digest"
input_2 = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
input_3 = "12345678901234567890123456789012345678901234567890123456789012345678901234567890"
message_input = input_3

Basic Operations

In [2]:
def XOR(i,j):
    return list(a^b for a,b in zip(i,j))
def AND(i,j):
    return [a and b for a, b in zip(i,j)]
def NOT(i): 
    return [int(not(a)) for a in i]
def MAJ(i,j,k): 
    return max([i,j,], key=[i,j,k].count)
def ROTR(x, n): 
    return x[-n:] + x[:-n]
def SHR(x, n): 
    return n * [0] + x[:-n]
def ADD(i, j):
    length = len(i)
    sums = list(range(length))
    #initial input needs a carry over bit as 0
    c = 0
    for a in range(length-1,-1,-1):
        #add the input bits with a double xor
        sums[a] = (i[a]^j[a])^c
        c = MAJ(i[a], j[a], c)
    return sums

Constant words calculation

In [3]:
# function for generating first N prime numbers
def generate_n_primes(N):
    primes  = []
    chkthis = 2
    while len(primes) < N:
        ptest    = [chkthis for i in primes if chkthis%i == 0]
        primes  += [] if ptest else [chkthis]
        chkthis += 1
    return primes
first_64_primes = generate_n_primes(64)

"""function for generating constant words that represent the first 32 bits 
of the fractional parts of the cube roots of the first 64 prime numbers"""

K=[]
for x in first_64_primes:
    cube_root = x ** (1./3.)
    fractional = cube_root % 1
    fractional = hex(int(fractional * (2**32)))
    K.append(fractional)

Stages:
1. Preprocessing
2. Hash Computation

### PREPROCESSING

Preprocessing consists of three steps:
<ol>
    <li>The first step: padding the message</li>
    <li>The second step: parsing the paded message</li>
    <li>Third step: computing the initial hash value</li>
</ol>

In [4]:
# before we start, we convert our message first into unicode values then into bits
def message_to_bit(message):
    #string characters to unicode values
    unicode_val = [ord(c) for c in message]
    #unicode values to 8-bit strings (removed binary indicator)
    byte_string = []
    for val in unicode_val:
        byte_string.append(bin(val)[2:].zfill(8))# add bit "0" in front if bit length less than 8
    #8-bit strings to list of bits as integers
    bits = []
    for byte in byte_string:
        for bit in byte:
            bits.append(int(bit))
    return bits

# append k zeros function (k = length - bits -1)
def append_zeros(bits, length):
    l = len(bits)
    for i in range(l, length):
        bits.append(0)
    return bits                                                 # PADDING

                        #The purpose is to create a padded message that is a multiple of 512.
                                    #1. We start by converting our message to bits.
                                    #2. Caculate its length and express it in bits.
                                    #3. The result of the padded message will be: 
                #The value of the message M in bits + 1 + k zero bits + 64-bit value of the length of the message.

def padded_message(message):
    # convert message into bits
    bits = message_to_bit(message)
    #calculate the message length as a 64-bit block 
    message_len = [int(b) for b in bin(len(bits))[2:].zfill(64)]
    if len(bits) < 448:
        #append single 1
        bits.append(1)
        #append k zeros until it is congruent with 448mod512 bits
        bits = append_zeros(bits, 448)
        #add the 64 bits representing the length of the message
        bits = bits + message_len
        #return the block
        return [bits]
    elif 448 <= len(bits) <= 512:
        bits.append(1)
        #move to next message block
        bits = append_zeros(bits, 1024)
        #replace the last 64 bits of the multiple of 512 with the original message length
        bits[-64:] = message_len
        #return it in 512 bit chunks
        return parser(bits, 512)
    else:
        bits.append(1)
        # loop until multiple of 512
        while len(bits) % 512 != 0:
            bits.append(0)
        bits[-64:] = message_len
    return parser(bits, 512)

"""            If the message is shorter than 448 bits, append the 1 bit , add k 0 (until it reaches 448 bits) 
                                            and then append the length expressed in bits. 
             If message's length is between 448 and 512 bits, append the 1 bit then add k 0 (until it reaches 1024 bits),
                                then replace the last 64 bits with the length of the message.
          If the message is longer than 512 bits, append the 1 bit, then append 0 until we reach the nearest multiple of 512, 
                                then replace the last 64 bits with the length of the message
"""


                                                    # PARSING

                    #While parsing we express our 512 bit padded message into 16 blocks with 32-bit each
# define the parser function
def parser(bits, block_length=8):
    parsed = []
    for i in range(0, len(bits), block_length):
        parsed.append(bits[i:i+block_length])
    return parsed
# parse the message in 32-bit blocks
for block in padded_message(message_input):
    w = parser(block, 32)
    
print("Number of M-bit blocks: ",len(w))
print("Number of bits per block: ",len(w[0]))



                                        # SETTING THE INITIAL HASH VALUE

                    #The first initial hash values are obtained by taking the first thirty-two bits 
                    #of the fractional parts of the square roots of the first eight prime numbers.

initial_hash_list=[]
# generate first 8 prime numbers
first_8_primes = generate_n_primes(8)
print("First 8 prime numbers: ", first_8_primes)
# calculate the fractional parts of the square roots of the first eight prime numbers
for i in first_8_primes:
    square_root = i ** (1./2.)
    fractional = square_root % 1
    fractional = hex(int(fractional * (2**32)))
    initial_hash_list.append(fractional)
print("Initial Hash Value: ", initial_hash_list)

Number of M-bit blocks:  16
Number of bits per block:  32
First 8 prime numbers:  [2, 3, 5, 7, 11, 13, 17, 19]
Initial Hash Value:  ['0x6a09e667', '0xbb67ae85', '0x3c6ef372', '0xa54ff53a', '0x510e527f', '0x9b05688c', '0x1f83d9ab', '0x5be0cd19']


### HASH COMPUTATION

Hash Computation consists of four steps:
<ol>
    <li>The first step: preparing the message schedule</li>
    <li>The second step: initializing the eight working variables</li>
    <li>The third step: performing 64 iterations of the compression function</li>
    <li>The fourth step: calculating the intermediate hash value</li>
</ol>

## Hash computation

In [5]:
"""Before we start with the hash computation stage we first initiliaze the constants and initial hash value 
and convert them in bits"""
# insert zeros at the beginning
def zeros_insert(bits,length):
    l = len(bits)
    while l < length:
        bits.insert(0, 0)
        l = len(bits)
    return bits 

def init(values):
    #convert from hex to python binary string (with cut bin indicator ('0b'))
    binaries = [bin(int(v, 16))[2:] for v in values]
    #convert from python string representation to a list of 32 bit lists
    words = []
    for binary in binaries:
        word = []
        for b in binary:
            word.append(int(b))
        words.append(zeros_insert(word, 32))
    return words

h0, h1, h2, h3, h4, h5, h6, h7 = init(initial_hash_list)
k = init(K)



                                        #PREPARE THE MESSAGE SCHEDULE

                        #we start by adding to our initial 16 parsed blocks 48 blocks
for block in padded_message(message_input):
    w = parser(block, 32)
    for t in range(48):
        w.append(32 * [0])
    for t in range(16, 64):
        s0 = XOR(XOR(ROTR(w[t-15], 7), ROTR(w[t-15], 18)), SHR(w[t-15], 3) ) 
        s1 = XOR(XOR(ROTR(w[t-2], 17), ROTR(w[t-2], 19)), SHR(w[t-2], 10))
        w[t] = ADD(ADD(ADD(w[t-16], s0), w[t-7]), s1)
 


                                        #INITIALIZE THE EIGHT WORKING VARIABLES
        
                #we assign the working variables with the (i-1)st hash value from the initial hash values list
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    f = h5
    g = h6
    h = h7
    
    
    
                                        #COMPRESSION FUNCTION MAIN LOOP

            #Perform 64 iterations of the compression function that is going to update the variables (a, b,…, h), 
    #with the help of two temporary words created using choose Ch and majority Maj functions, the constant words Kt{256}, 
                        #the message schedule Wt, and the eight working variables themselves
    for j in range(64):
        S1 = XOR(XOR(ROTR(e, 6), ROTR(e, 11)), ROTR(e, 25) )
        ch = XOR(AND(e, f), AND(NOT(e), g))
        temp1 = ADD(ADD(ADD(ADD(h, S1), ch), k[j]), w[j])
        S0 = XOR(XOR(ROTR(a, 2), ROTR(a, 13)), ROTR(a, 22))
        m = XOR(XOR(AND(a, b), AND(a, c)), AND(b, c))
        temp2 = ADD(S0, m)
        h = g
        g = f
        f = e
        e = ADD(d, temp1)
        d = c
        c = b
        b = a
        a = ADD(temp1, temp2)
        
        
        
                                    #CALCULATE THE INTERMEDIATE HASH VALUE
            
                            #Add the compressed chunk to the current hash value
    h0 = ADD(h0, a)
    h1 = ADD(h1, b)
    h2 = ADD(h2, c)
    h3 = ADD(h3, d)
    h4 = ADD(h4, e)
    h5 = ADD(h5, f)
    h6 = ADD(h6, g)
    h7 = ADD(h7, h)
    
    

                                    #PRODUCE THE FINAL HASH VALUE (big-endian)
        
                    #256-bit message digest of the message, M, is the union of all hashes

# function to convert the list of 32 bits into a hexadecimal value
def bit_to_hex(value):
    value = ''.join([str(x) for x in value])
    binaries = []
    for d in range(0, len(value), 4):
        binaries.append('0b' + value[d:d+4])
    hexes = ''
    for b in binaries:
        hexes += hex(int(b ,2))[2:]
    return hexes

digest_bit = ''
digest = ''
for hash_value in [h0, h1, h2, h3, h4, h5, h6, h7]:
    digest_bit += ''.join([str(x) for x in hash_value])
    digest += bit_to_hex(hash_value)
print("The hash value of the message ", message_input, " produced from our algorithm is: ", digest)

The hash value of the message  12345678901234567890123456789012345678901234567890123456789012345678901234567890  produced from our algorithm is:  f371bc4a311f2b009eef952dd83ca80e2b60026c8e935592d0f9c308453c813e


### Message results for each step

In [6]:
print("1) Message in bits: ", ''.join([str(x) for x in message_to_bit(message_input)]))

pd_list=[j for i in padded_message(message_input) for j in i]
print("\n2) Padded message: ", ''.join([str(x) for x in a]))

print("\n3) Digest in bits: ", digest_bit)

print("\n4) The hash value of the message: ", digest)

1) Message in bits:  0011000100110010001100110011010000110101001101100011011100111000001110010011000000110001001100100011001100110100001101010011011000110111001110000011100100110000001100010011001000110011001101000011010100110110001101110011100000111001001100000011000100110010001100110011010000110101001101100011011100111000001110010011000000110001001100100011001100110100001101010011011000110111001110000011100100110000001100010011001000110011001101000011010100110110001101110011100000111001001100000011000100110010001100110011010000110101001101100011011100111000001110010011000000110001001100100011001100110100001101010011011000110111001110000011100100110000

2) Padded message:  11110000111100000111011110010000

3) Digest in bits:  1111001101110001101111000100101000110001000111110010101100000000100111101110111110010101001011011101100000111100101010000000111000101011011000000000001001101100100011101001001101010101100100101101000011111001110000110000100001000101001111001000000100111110

4) Th

## Test

In [7]:
import hashlib
encoded_my_name=message_input.encode()
digest_hlib = hashlib.sha256(encoded_my_name).hexdigest()
print("The hash value of the message ", message_input, "produced by the hashlib algorithm is: ", digest_hlib)

The hash value of the message  12345678901234567890123456789012345678901234567890123456789012345678901234567890 produced by the hashlib algorithm is:  f371bc4a311f2b009eef952dd83ca80e2b60026c8e935592d0f9c308453c813e


In [8]:
# Code Reference
#https://www.codespeedy.com/perform-xor-on-two-lists-in-python/
#https://blog.boot.dev/cryptography/how-sha-2-works-step-by-step-sha-256/
#https://stackoverflow.com/questions/16312730/comparing-two-lists-and-only-printing-the-differences-xoring-two-lists
#https://stackoverflow.com/questions/1628949/to-find-first-n-prime-numbers-in-python